In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller


## Importation des données

In [8]:
df = pd.read_csv('data.csv', sep=';')

In [9]:
# Conversion de la colonne 'date' en format datetime
df['annee'] = pd.to_datetime(df['annee'], format='%Y')

# Définir la date comme index
df.set_index('annee', inplace=True)

In [11]:
#df['annee'] = pd.to_datetime(df['annee'], format='%Y')

df.to_csv('seriTemp.csv', sep=';', index=True)

In [9]:
df.columns

Index(['production', 'Superficie (1000 ha)', 'Precipitation (mm)',
       'Subvention( million fcfa)', 'prix (fcfa)'],
      dtype='object')

## *Nettoyage  des données*

In [12]:
# Nettoyage :
def nettoyage(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            # Supprimer les espaces
            df[col] = df[col].str.strip()
            # Remplacer les virgules par des points
            df[col] = df[col].str.replace(',', '.', regex=False)
            df[col] = df[col].str.replace(' ', '', regex=False)
            # Convertir en numérique
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

df = nettoyage(df)
# Vérification finale
print(df.dtypes)
print(df.head())

production                   float64
Superficie (1000 ha)           int64
Precipitation (mm)           float64
Subvention( million fcfa)      int64
prix (fcfa)                  float64
dtype: object
            production  Superficie (1000 ha)  Precipitation (mm)  \
annee                                                              
2023-01-01   166558.86                   462              820.36   
2022-01-01   168736.10                   618              939.28   
2021-01-01   209015.04                   595              843.13   
2020-01-01   206837.80                   556              952.49   
2019-01-01   191597.12                   580              913.41   

            Subvention( million fcfa)  prix (fcfa)  
annee                                               
2023-01-01                     100000       554.16  
2022-01-01                      72800       492.90  
2021-01-01                      12700       431.04  
2020-01-01                      14000       445.02  
2019-0

## Construction du modèle : production = subventions+prix+precipitaions+Superficie

*Les variables*

In [13]:
X = df[['Superficie (1000 ha)', 'Subvention( million fcfa)','prix (fcfa)','Precipitation (mm)']]  # variables explicatives
X = sm.add_constant(X)  # ajoute une constante (β0)
y = df['production']     # variable cible


### *le modele*

In [14]:
model = sm.OLS(y, X).fit()


In [ ]:
model.summary()


## Test de Stationnarité

In [19]:

# Exemple sur une série nommée "production"
result = adfuller(df['production'])

In [20]:
result

(np.float64(-0.49052402622876373),
 np.float64(0.8938524417828728),
 10,
 40,
 {'1%': np.float64(-3.6055648906249997),
  '5%': np.float64(-2.937069375),
  '10%': np.float64(-2.606985625)},
 np.float64(912.6093656765423))

In [23]:
def test_stationarite(serie):
    print("=== Test ADF ===")
    result_adf = adfuller(serie.dropna())
    
    #print(result_adf)
    print(f"ADF Statistic: {result_adf[0]}, p-value: {result_adf[1]}")

In [24]:
test_stationarite(df['production'])

=== Test ADF ===
ADF Statistic: -0.49052402622876373, p-value: 0.8938524417828728


In [15]:
from statsmodels.tsa.stattools import coint

# Exemple : production et prix
score, pvalue, _ = coint(df['production'], df[['Superficie (1000 ha)', 'Subvention( million fcfa)','prix (fcfa)','Precipitation (mm)']] )
print('p-value de co-intégration :', pvalue)


p-value de co-intégration : 0.9918748746835239


In [16]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen


In [17]:
# paramètre det_order = -1 (automatique), k_ar_diff = nombre de retards
johansen_test = coint_johansen(df, det_order=0, k_ar_diff=1)


In [18]:
trace_stat = johansen_test.lr1      # Statistiques du test Trace
crit_values = johansen_test.cvt     # Valeurs critiques à 90%, 95%, 99%

for i in range(len(trace_stat)):
    print(f"Hypothèse r <= {i} : Stat = {trace_stat[i]:.2f}, Critères : {crit_values[i]}")


Hypothèse r <= 0 : Stat = 126.78, Critères : [65.8202 69.8189 77.8202]
Hypothèse r <= 1 : Stat = 49.28, Critères : [44.4929 47.8545 54.6815]
Hypothèse r <= 2 : Stat = 28.69, Critères : [27.0669 29.7961 35.4628]
Hypothèse r <= 3 : Stat = 11.79, Critères : [13.4294 15.4943 19.9349]
Hypothèse r <= 4 : Stat = 1.30, Critères : [2.7055 3.8415 6.6349]
